In [1]:
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import requests

load_dotenv()

dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]
erAppId = os.environ["APP_ID"]

common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(dbUrl))
uid = common.authenticate(dbName, dbUsername, dbPassword, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(dbUrl))

# Data Handling Functions
def id_field_into_1value(jsonFile):
    for index, line in enumerate(jsonFile):
        for item in line:
            if type(line[item]) == list:
                jsonFile[index][item] = jsonFile[index][item][1]

df = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order.log',
    'search_read',
    [],
    {'fields': [] })

id_field_into_1value(df)

df = pd.DataFrame(df)

# Create a dictionary for currency conversion
currency_mapping = {
    "ARS": "ARS",
    "CLP": "CLP",
    "PEN": "PEN",
    "COP": "COP",
    "MXN": "MXN",
    "USD": "USD",
    "BRL": "BRL",
    "EUR": "EUR",
}

# Send a GET request to the API
url = f'https://openexchangerates.org/api/latest.json?app_id={erAppId}'
headers = {"accept": "application/json"}

# manage api response
response = requests.get(url, headers=headers)
data = response.json()

# Extract exchange rates from the data
rates = data['rates']

# map only the used currencies exchnage rates based on USD
usd_er = {currency_mapping[key]: rates[key] for key in currency_mapping.keys()}

# Extract the currency from the 'pricelist_id' column
# df['currency_id'] = df['currency_id'].str.extract(r'\((.*?)\)', expand=False)

# Map the full currency names to their three-letter ISO codes
df['currency_id'] = df['currency_id'].map(currency_mapping)

# Map the exchange rates to a new column based on 'pricelist_id'
df['exchange_rate'] = df['currency_id'].map(usd_er)

# Divide 'recurring_monthly' and 'amount_signed' by 'exchange_rate' to convert to USD
df['recurring_monthly_usd'] = round(df['recurring_monthly'].astype(float) / df['exchange_rate'], 2)
df['amount_signed_usd'] = round(df['amount_signed'].astype(float) / df['exchange_rate'], 2)


DATABASE_CONNECTION_URI = os.environ["DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

# insert the dataframe into the postgres table
df.to_sql('od_sale_order_changes_log', con=engine, if_exists='replace', index=False)

141